# CET 324 Authentication System

__Notes__    
Used bcrypt for reasons given by: https://blog.cloudflare.com/keeping-passwords-safe-by-staying-up-to-date/    
Use AES for encryption of token. Create a class with a defined key for the AES encryption.    
Use Asymmetric encryption to encrypt the private shared key for Symmetric encryption

In [1]:
from datetime import date
from dateutil.relativedelta import relativedelta
import hashlib, io, hmac
from cryptography.fernet import Fernet
import csv
import bcrypt

In [2]:
class User:
    def __init__(self, username, password, permission):
        self.username = username
        self.password = password
        self.permission = permission
        
    def requestAccessToken(self, system):
        accessToken = AccessToken(self, system.systemName)
        return accessToken.generateToken()
    
    def userAccountPage(self, system):
        print(self.username + "\'s Account Page")
        print("___________________")
        while 1: 
            print("1. Request token")
            print("0. Logout")
            
            user_command = int(input("Enter your command: "))
            
            #if else statement to carry out user command 
            if user_command == 1:
                new_token = self.requestAccessToken(system)
                print(new_token)
            elif user_command == 0:
                print("Logging Out")
                break
            else: 
                print("Invalid Command")

In [3]:
class AccessToken:
    def __init__(self, user, systemName):
        self.user = user
        self.systemName = systemName
        self.date = date.today()
        
    def calcExpiryDate(self):
        #add 6 months to current date to calculate expiry date 
        return self.date + relativedelta(months=6)
    
    def generateToken(self):
        #need to add code to generate a secure token 
        self.user.username + self.user.permission + str(self.date) + str(self.calcExpiryDate()) + self.createDigitalSignature()

    def createDigitalSignature(self):
        #need to add code to create a digital signature 
        return "securelyencrypted"

In [4]:
class System:    
    #set default permission to 'u' (user) for creating a new account
    DEFAULT_PERMISSION = "u"
   
    def __init__(self, systemName):
        self.systemName = systemName
        
        
    def login(self, username, password): 
        with open(self.systemName + 'users.csv', newline='') as userscsv:
            fieldnames = ['username', 'password', 'permission']
            reader = csv.DictReader(userscsv, fieldnames=fieldnames)
            for row in reader:
                if row['username'] == username:
                    #convert password to bytes 
                    enteredPassword = password.encode('utf-8')
                    
                    savedPassword = row['password']
                
                    #check password 
                    if bcrypt.checkpw(enteredPassword, savedPassword.encode('utf-8')):
                        return User(row['username'], row['password'], row['permission'])
                    
            #return null if there is no match
            return None
    
            
    def hashpassword(self, plainPassword):
        #convert password to array of bytes 
        bytes = plainPassword.encode('utf-8')
        
        #generate salt 
        salt = bcrypt.gensalt()
        
        #hash and salt password 
        hashpassword = bcrypt.hashpw(bytes, salt)
        
        return hashpassword
        
    def loginWithToken():
        #need to implement code to check token is valid and return bool 
        return "loginWithTokenCalled"
    
    def createAccount(self, user):
        with open(self.systemName + 'users.csv', 'a', newline='') as userscsv:
            fieldnames = ['username', 'password', 'permission']
            writer = csv.DictWriter(userscsv, delimiter=',', quotechar='|', 
                                   quoting=csv.QUOTE_MINIMAL, fieldnames=fieldnames)
            hashedpw = self.hashpassword(user.password)
        
            writer.writerow({'username': user.username, 'password': hashedpw.decode('utf-8'), 'permission': user.permission})

    def read_csv(self):
        with open(self.systemName + 'users.csv', newline='') as userscsv:
            fieldnames = ['username', 'password']
            reader = csv.DictReader(userscsv, fieldnames=fieldnames)
            for row in reader:
                print("", row['username'], row['password'], row['permission'])
            
    def display_main_menu(self):
        while 1: 
            print(self.systemName)
            print("___________________")
            print("Main Menu")
            print("1. Login")
            print("2. Create Account")
            print("0. Exit")
            
            user_command = int(input("Enter your command: "))
            
            #if else statement to carry out user command 
            if user_command == 1:
                username = input("Enter Username: ")
                password = input("Enter password: ")
                user = self.login(username,  password)
                if user is None:
                    print("Incorrect username or password")                    
                else:
                    user.userAccountPage(self)
            elif user_command == 2:
                username = input("Enter new Username: ")
                password = input("Enter new password: ")
                self.createAccount(User(username, password, self.DEFAULT_PERMISSION))
            elif user_command == 0:
                print("Exiting")
                break
            else: 
                print("Invalid Command")

In [6]:
system = System("Super Secure System")

system.display_main_menu()

Super Secure System
___________________
Main Menu
1. Login
2. Create Account
0. Exit
Enter your command: 2
Enter new Username: userr1
Enter new password: password
Super Secure System
___________________
Main Menu
1. Login
2. Create Account
0. Exit
Enter your command: 1
Enter Username: userr1
Enter password: password
userr1's Account Page
___________________
1. Request token
0. Logout
Enter your command: 1
userr1u2024-05-022024-11-02securelyencrypted
1. Request token
0. Logout
Enter your command: 0
Logging Out
Super Secure System
___________________
Main Menu
1. Login
2. Create Account
0. Exit
Enter your command: 0
Exiting
